In [ ]:
import gzip
import h5py
import librosa
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import soundfile as sf
from collections import OrderedDict, Counter

%matplotlib inline

dcase_dir = '/beegfs/vl1019/waspaa2019_data/BirdVox-DCASE-20k_h5'

settings = {
    "fmin": 2000,
    "fmax": 11025,
    "hop_length": 32,
    "n_fft": 1024,
    "n_mels": 128,
    "sr": 22050,
    "win_length": 256,
    "window": "hann",
    "T": 0.1,
    "alpha": 1.0,
    "delta": 0.0,
    "r": 1.0,
    "eps": 1e-6,
    "lower_sf_bin": 15,
    "upper_sf_bin": 90,
    "padding": 10}

# greater padding for BirdVox-DCASE because there is no signal duplication around boundaries
settings["padding"] = 100
coni_thresholds_path = '/beegfs/vl1019/waspaa2019_data/coni-knight-thresholds_v-2.csv'
coni_thresholds = pd.read_csv(coni_thresholds_path)

distance_list = coni_thresholds['nominal_distance'].unique()
bird_id_list = coni_thresholds['bird_id'].unique()

import tqdm

initial_chars =\
    [str(x) for x in range(10)] + [chr(ord('a') + x) for x in range(6)]



log_pcen_max_list = []
log_pcen_avg_list = []
melsf_max_list = []
melsf_avg_list = []



# Loop over 16 hexadecimal prefixes in BirdVox-DCASE-20k
for initial_char in tqdm.tqdm(initial_chars):
    h5_name = "_".join(["BirdVox-DCASE-20k_pcen", initial_char]) + ".h5"
    h5_path = os.path.join(dcase_dir, h5_name)
    
    # Open HDF5 file
    with h5py.File(h5_path, 'r') as f:
        log_pcen_max_list.append(np.max(np.log1p(
            f["pcen"][:, settings["lower_sf_bin"]:settings["upper_sf_bin"], padding:-padding]), axis=1));
        log_pcen_avg_list.append(np.mean(np.log1p(
            f["pcen"][:, settings["lower_sf_bin"]:settings["upper_sf_bin"], padding:-padding]), axis=1));
        melsf_max_list.append(np.max(np.maximum(
            0, np.diff(np.log1p(f["mel"][:, settings["lower_sf_bin"]:settings["upper_sf_bin"], padding:-padding]))), axis=1));
        melsf_avg_list.append(np.mean(np.maximum(
            0, np.diff(np.log1p(f["mel"][:, settings["lower_sf_bin"]:settings["upper_sf_bin"], padding:-padding]))), axis=1));
        
feature_matrix = {
    "log_pcen_max": np.concatenate(log_pcen_max_list, axis=0),
    "log_pcen_avg": np.concatenate(log_pcen_avg_list, axis=0),
    "melsf_max": np.concatenate(melsf_max_list, axis=0),
    "melsf_avg": np.concatenate(melsf_avg_list, axis=0)
}

feature_strs = [
    "melsf_avg", "melsf_max",
    "log_pcen_avg", "log_pcen_max"]
hop_duration = pcen_settings["hop_length"] / pcen_settings["sr"]
mtbf_dict = {feature_str: [] for feature_str in feature_strs}

for _, row in tqdm.tqdm(coni_thresholds.iterrows()):
    for feature_str in feature_strs:
        n_false_alarms = np.sum(
            row[feature_str] < (feature_matrix[feature_str] - np.min(feature_matrix[feature_str], axis=1)[:, np.newaxis]))
        n_samples = np.prod(feature_matrix[feature_str].shape)
        mtbf = hop_duration * (n_samples/(n_false_alarms))
        mtbf_dict[feature_str].append(mtbf)

mtbf_dict["bird_id"] = coni_thresholds["bird_id"]
mtbf_dict["nominal_distance"] = coni_thresholds["nominal_distance"]

In [ ]:
plt.rcParams["font.family"] = "serif"


x_distances = np.unique(mtbf_df["nominal_distance"])

model_colors = [
    "#CB0003", 
    "#E67300",
    "#990099",
    "#0000B2",
    "#009900",
    '#008888',
    '#888800',
    '#555555',
]

mtbf_df = pd.DataFrame(mtbf_dict)
feature_colors = {
    "melsf_max": "#0000B2",
    "melsf_avg": "#E67300",
    "log_pcen_max": "#009900",
}
feature_alphas = {
    "#0000B2": 0.33,
    "#E67300": 0.5,
    "#009900": 0.5
    
}
feature_markers = {
    "melsf_avg": "-v",
    "melsf_max": "-s",
    "log_pcen_max": "-o",
}
feature_labels = {
    "melsf_avg": "Averaged spectral flux",
    "melsf_max": "Max-pooled spectral flux",
    "log_pcen_avg": "Averaged PCEN",
    "log_pcen_max": "Max-pooled PCEN",
}

plt.figure(figsize=(6, 4))
plt.title('Bioacoustic detection of Common Nighthawk', size=12)

# Loop over features
for feature_str in ["log_pcen_max", "melsf_max", "melsf_avg"]:
    
    median_mtbf_list = []
    lower_mtbf_list = []
    upper_mtbf_list = []
    
    # Loop over distances
    for x_distance in x_distances:
        
        distance_df = mtbf_df[
            (mtbf_df["nominal_distance"]==x_distance)]
        median_mtbf_list.append(
            np.median(distance_df[feature_str]))
        lower_mtbf_list.append(
            np.quantile(distance_df[feature_str], 0.25))
        upper_mtbf_list.append(
            np.quantile(distance_df[feature_str], 0.75))
        

    plt.plot(
        x_distances,
        np.log10(median_mtbf_list),
        feature_markers[feature_str],
        markersize=10.0,
        linewidth=2.0,
        color=feature_colors[feature_str],
        label=feature_labels[feature_str])
    plt.fill_between(
        x_distances, 
        np.log10(lower_mtbf_list), np.log10(upper_mtbf_list),
        color=feature_colors[feature_str],
        alpha=feature_alphas[feature_colors[feature_str]])
        
plt.legend(prop={'size': 11})
yticks = np.array([0.1, 0.2, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0, 50.0, 100.0, 200.0])

plt.gca().set_xticks(np.arange(0, 501, 100))
plt.gca().set_xticklabels(np.arange(0, 501, 100), size=11)
plt.gca().set_xticks(np.arange(0, 501, 50), minor=True)
plt.gca().set_yticks(np.log10(yticks))
plt.gca().set_yticklabels(yticks, size=11);
plt.grid(which='minor', linestyle='--', alpha=1.0);
plt.grid(which='major', linestyle='--', alpha=1.0);

plt.xlim(0, 500)
plt.ylim(np.log10(0.5), np.log10(200.0))

plt.xlabel("CONI-Knight dataset: distance between sensor and source (m)", size=12)
plt.ylabel("BirdVox-DCASE-20k dataset: mean time\nbetween false alarms at half recall (s)", size=12)


ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


plt.savefig('lostanlen_waspaa2019_coni_mtbfa-50_semilogy.eps', bbox_inches='tight')
plt.savefig('lostanlen_waspaa2019_coni_mtbfa-50_semilogy.png', bbox_inches='tight', dpi=500)